### Optimizing Temporary Housing Deployment to Reduce U.S. Homelessness 
Nauman Sohani, Yannan Tuo, Charlie Nitschelm

In [219]:
# Load packages
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [220]:
# Import datasets
# from <data_source> import *
%whos

Variable                Type         Data/Info
----------------------------------------------
CoCPopulations          dict         n=3
Factory                 tupledict    {0: <gurobi.Var FactoryLo<...>cations[2] (value -0.0)>}
FactoryCost             int          1000
GRB                     type         <class 'gurobipy._grb.GRB'>
NumCities               int          3
NumFactories            int          3
THU                     tupledict    {0: <gurobi.Var THUQuanti<...>yPerCity[2] (value 0.0)>}
THUClosestFactoryCity   tupledict    {(0, 0): <gurobi.Var Clos<...>actory[2,2] (value 0.0)>}
THUCost                 int          50000
THUShippingBase         int          100
THUShippingPerMile      int          1
TotalBudget             int          50000000
budgetConstr            QConstr      <gurobi.QConstr BudgetConstr>
cities                  dict_keys    dict_keys(['SF', 'Boston', 'NYC'])
city_distance           dict         n=3
closestFactoryConstr    tupledict    {0: <gurobi.Con

In [221]:
# Set up Gurobi environment
env = gp.Env(empty=True)
env.setParam('OutputFlag', 0)
env.start()

# Initialize the model
m = gp.Model(env=env)

In [222]:
# High level variables + assumptions
# Total budget allocated for uplift units
TotalBudget = 50000000 # 10000000

# Uplift unit specification assumptions
# Uplift Factory Spec 
FactoryCost = 1000

# Uplift THU Spec 
THUCost = 50000

# Uplift Shipping Spec 
THUShippingBase = 100
THUShippingPerMile = 1

# Uplift Operational Spec 
# THU_to_Factory_Max_Distance = 50

In [223]:
# City distance data. What to use as baseline? Generate dictionary for all cities? (500-ish)
city_distance = {
                 "SF": {"SF": 0, "Boston": 3000, "NYC": 2900}, 
                 "Boston": {"SF": 3000, "Boston": 0, "NYC": 215},
                 "NYC": {"SF": 2900, "Boston": 215, "NYC": 0}
                 }

cost_matrix = city_distance
# use google maps api distance between cities
# regex: parse out first city/county name to use as reference
# look for comma, slash, "Statewide", "CoC", "County," or "Counties" as separator
# for regional locations, eg "Southeast Arkansas" "Northeast Alabama" just use default queried location (eg community college)
# regex: is it "county/counties" or a city
# tack on state (column 1), ie "champaign, IL" in query

# get centroid from geopandas = get latitude/longitude -> get 




In [224]:
# CoC Data/homeless population
CoCPopulations = {"SF": 8300, "Boston": 6000, "NYC": 350000}

# Number of cities
NumCities = 3

# Number of factory locations available (equivalent to cities)
# todo: rename var
NumFactories = 3

# ordered dict for version < 3.7; assumed same order in version >= 3.7
indices = range(len(CoCPopulations))
cities = CoCPopulations.keys()
indexToCityDict = dict(zip(indices, cities))
def indexToCity(index):
    return indexToCityDict[index]

print(indexToCityDict)

{0: 'SF', 1: 'Boston', 2: 'NYC'}


In [225]:
# Decision variables
# How many homes to place per city
THU = m.addVars((t for t in range(0, NumCities)), lb=0, name="THUQuantityPerCity")

# Where to place factories
Factory = m.addVars((t for t in range(0, NumFactories)), vtype=GRB.BINARY, name="FactoryLocations")

# What factory services the THU
# THUClosestFactoryCity = m.addVars((t for t in range(0, NumCities)), lb=0, name="ClosestFactory")

# city-factory combo
# THUClosestFactoryCity = m.addVars((t for t in range(0, NumCities)), lb=0, name="ClosestFactory")
THUClosestFactoryCity = m.addVars(((i, j) for i in range(0, NumCities) for j in range(0, NumFactories)), vtype=GRB.BINARY, name="ClosestFactory")
# THUClosestFactoryCity = m.addVars(((i, j) for i in range(0, NumCities) for j in range(0, NumFactories)), lb=0, ub = 1, name="ClosestFactory")


In [226]:
# Constraints
# Budget constraint
# sum cost of all the THUs + 
# sum cost of all the factories + 
# sum cost of transportation of THUs from all the factories = # produced in city * closest factory * distance to factory * shipping cost per mile
budgetConstr = m.addConstr((sum(THU[cityIndex]*THUCost for cityIndex in range(0, NumCities)) + 
                              FactoryCost*sum(Factory[factory] for factory in range(NumFactories)) +
                              sum(THU[cityIndex]*THUClosestFactoryCity[(cityIndex, factory)]*city_distance[indexToCity(cityIndex)][indexToCity(factory)]*THUShippingPerMile 
                              for cityIndex in range(NumCities) for factory in range(NumFactories))) <= TotalBudget, name='BudgetConstr')

# Unhoused population (allocate no more than # unhoused)
popConstr = m.addConstrs(THU[cityIndex] <= CoCPopulations[indexToCity(cityIndex)] for cityIndex in range(0, NumCities))

# There must be at least one factory in total
factoryConstr = m.addConstr(sum(Factory[factory] for factory in range(0, NumCities)) >= 1)

# Each city can only have one closest city for the factory
closestFactoryConstr = m.addConstrs(sum(THUClosestFactoryCity[(cityIndex, factory)] for factory in range(NumCities)) == 1 for cityIndex in range(NumCities))

# Closest factory for a city must be a selected factory 
factoryLimit = m.addConstrs(THUClosestFactoryCity[(cityIndex, factory)] <= Factory[factory] for cityIndex in range(NumCities) for factory in range(NumFactories))


In [227]:
# Ignore: old ref
# # fixes: create decision variable to set closest factory per city? (should naturally select closest factory)

# for cityIndex in range(NumCities):
#     nearestFactoryCity = findClosestFactory(indexToCity(cityIndex))
#     THU[cityIndex]*THUCost + THU[cityIndex]*city_distance[indexToCity(cityIndex)]*THUShippingPerMile
# m.addConstr(sum(costs) <= TotalBudget, name='BudgetConst')


# city * min_factory_distance * shipping_per_mile
# how to get min_factory_distance
# * min([city_distance[for every city] where factory_selected])
# factory_selected has to be true (assume 1)

# Budget
# budgetConstr = m.addConstr(sum(THU[cityIndex]*THUCost + 
#                                THU[cityIndex]*min(city_distance[indexToCity(cityIndex)])*THUShippingPerMile 
#                                for cityIndex in range(0, NumCities)) <= TotalBudget, name='BudgetConst')
# budgetConstr = m.addConstr(sum(THU[cityIndex]*THUCost + THU[cityIndex]*min(city_distance[indexToCity(factoryCityIndex)] for factoryCityIndex in range(0, NumCities) if Factory[factoryCityIndex] == True)*THUShippingPerMile for cityIndex in range(0, NumCities)) <= TotalBudget, name='BudgetConst'

# budgetConstr = m.addConstr(sum(THU[cityIndex]*THUCost + 
#                                THU[cityIndex]*city_distance[indexToCity(THUClosestFactoryCity[cityIndex])]*THUShippingPerMile 
#                                for cityIndex in range(0, NumCities)) <= TotalBudget, name='BudgetConstr')


# sum(THU[cityIndex]* cost * min(selected_or_not) for cost in cost matrix(for city))
# pick min of selected_or_not
# selected_or_not applies to all cities

# budgetConstr = m.addConstr(sum(THU[cityIndex]*THUCost + THU[cityIndex]*cost_matrix[cityIndex]*THUShippingPerMile for cityIndex in range(0, NumCities)) <= TotalBudget, name='BudgetConstr')


In [228]:
# Objective function
# Maximize number of housing units provided for unhoused populations (ie; minimize unhoused individiuals)
m.setObjective(sum(THU[city] for city in range(0, NumCities)), gp.GRB.MAXIMIZE)


In [229]:
# Update and write the model
m.update() # Update model parameters
# m.write("uplift.lp") # Write model to file

# Solve
m.optimize()

# Check model is not infeasible
if m.status == GRB.INFEASIBLE:
    print("model is infeasible")
print("\nObjective value: ", "%.2f" % m.getAttr("ObjVal"))



Objective value:  999.94


In [230]:
# Print solution
for i in THU:
    # if THU[i].x > 0:
        print('%s: %g' % (indexToCity(i), THU[i].x))

for i in Factory:
    # if THU[i].x > 0:
        print('%s: %g' % (indexToCity(i), Factory[i].x))


for i in range(NumCities):
    for j in range(NumFactories):
    # if THU[i].x > 0:
        print('source city: %s, factory city: %s, value: %g' % (indexToCity(i), indexToCity(j), THUClosestFactoryCity[(i, j)].x))

print("THU cost: ", sum(THU[cityIndex].x*THUCost for cityIndex in range(0, NumCities)))
print("Factory cost: " + str(FactoryCost*sum(Factory[factory].x for factory in range(NumFactories))))
print("THU ship cost: " + str(sum(THU[cityIndex].x*THUClosestFactoryCity[(cityIndex, factory)].x*city_distance[indexToCity(cityIndex)][indexToCity(factory)]*THUShippingPerMile for cityIndex in range(NumCities) for factory in range(NumFactories))))

SF: 999.94
Boston: 0
NYC: 0
SF: 1
Boston: 1
NYC: 1
source city: SF, factory city: SF, value: 1
source city: SF, factory city: Boston, value: -0
source city: SF, factory city: NYC, value: -0
source city: Boston, factory city: SF, value: -0
source city: Boston, factory city: Boston, value: 1
source city: Boston, factory city: NYC, value: -0
source city: NYC, factory city: SF, value: -0
source city: NYC, factory city: Boston, value: -0
source city: NYC, factory city: NYC, value: 1
THU cost:  49997000.0
Factory cost: 3000.0
THU ship cost: 0.0


#### Analysis of optimization results
beep boop